### Задание

Руководитель отдела сопровождения клиентов хочет получить информацию о том, насколько хорошо сотрудники его отдела (аккаунт-менеджеры) справляются с одной из своих основных задач – пролонгацией договоров с клиентами. От аналитика он хочет получить отчет о пролонгациях сотрудников за 2023 год. 

В компании используется два коэффициента пролонгации:

1.	Для проектов пролонгированных в первый месяц – отношении суммы отгрузки (Отгрузка – сумма из таблицы financial_data.csv ) проектов пролонгированных в первый месяц после завершения к сумме отгрузки последнего месяца реализации всех завершившихся в прошлом месяце проектов.
  
2.	Для проектов, пролонгированных во второй месяц – отношение суммы отгрузки проектов, пролонгированных во второй месяц к сумме отгрузки последнего месяца проектов, не пролонгированных в первый. 

То есть, если нам нужно понять, насколько хорошо менеджер пролонгировал в мае, необходимо посчитать:

1.	Сумму отгрузки проектов, завершившихся в апреле (за апрель) и сумму отгрузки тех проектов завершившихся в апреле, у которых есть отгрузка в мае (за май). Коэффициент – отношение второй суммы к первой.
  
2.	Сумму проектов, завершившихся в марте, у которых нет отгрузки в апреле (за март) и сумму отгрузки тех проектов, завершившихся в марте, у которых нет отгрузки в апреле но есть в мае (за май). Коэффициент – отношение второй суммы к первой. 


Имеются два набора данных:

1.	prolongations.csv
	- id – id проекта
	- month – последний месяц реализации проекта
	- AM – ФИО ответственного аккаунт-менеджера (данные первичны по отношению к financial_data)

2.	financial_data.csv:
    - id – id проекта
	- Причина дубля – причина, почему строки с одним и тем же id встречаются несколько раз
	- Колонки с названием месяца – сумма отгрузки проекта в данный месяц.
В данной колонке могут также встречаться такие значения: ‘в ноль’ –  отгрузка проекта в данном месяце равна 0, значит для коэффициента пролонгации нужно взять отгрузку предыдущего месяца (только если все части оплаты равны 0); ‘стоп’ – проект закончился до истечения срока договора, если у проекта есть “стоп” в последний месяц реализации или ранее, то такой проект исключаем из пролонгаций; ‘end’ – аналогично ‘стоп’	
    - Account – ФИО ответственного аккаунт-менеджера


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [9]:
prol  = pd.read_csv("prolongations.csv", header=0, 
                         names=['project_id','last_month','account_name'])
fin  = pd.read_csv("financial_data.csv")

In [11]:
prol['last_month'] = prol['last_month'].str.capitalize()

In [12]:
prol['last_month'].unique()

array(['Ноябрь 2022', 'Декабрь 2022', 'Январь 2023', 'Февраль 2023',
       'Март 2023', 'Апрель 2023', 'Май 2023', 'Июнь 2023', 'Июль 2023',
       'Август 2023', 'Сентябрь 2023', 'Октябрь 2023', 'Ноябрь 2023',
       'Декабрь 2023'], dtype=object)

In [6]:
fin_df.columns

Index(['id', 'Причина дубля', 'Ноябрь 2022', 'Декабрь 2022', 'Январь 2023',
       'Февраль 2023', 'Март 2023', 'Апрель 2023', 'Май 2023', 'Июнь 2023',
       'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023',
       'Ноябрь 2023', 'Декабрь 2023', 'Январь 2024', 'Февраль 2024',
       'Account'],
      dtype='object')

In [8]:
non_month_cols = ["id", "Причина дубля", "Account"]
month_cols = [col for col in fin_df.columns if col not in non_month_cols]
month_cols

['Ноябрь 2022',
 'Декабрь 2022',
 'Январь 2023',
 'Февраль 2023',
 'Март 2023',
 'Апрель 2023',
 'Май 2023',
 'Июнь 2023',
 'Июль 2023',
 'Август 2023',
 'Сентябрь 2023',
 'Октябрь 2023',
 'Ноябрь 2023',
 'Декабрь 2023',
 'Январь 2024',
 'Февраль 2024']

In [3]:
prolong_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   project_id    477 non-null    int64 
 1   last_month    477 non-null    object
 2   account_name  477 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.3+ KB


In [4]:
prolong_df.head()

,project_id,last_month,account_name
0,42,ноябрь 2022,Васильев Артем Александрович
1,453,ноябрь 2022,Васильев Артем Александрович
2,548,ноябрь 2022,Михайлов Андрей Сергеевич
3,87,ноябрь 2022,Соколова Анастасия Викторовна
4,429,ноябрь 2022,Соколова Анастасия Викторовна


In [13]:
prolong_df['project_id'].value_counts()

project_id
633     6
154     6
547     5
798     5
600     5
       ..
717     1
976     1
1012    1
1001    1
868     1
Name: count, Length: 313, dtype: int64

In [14]:
prolong_df[prolong_df['project_id']==633]

,project_id,last_month,account_name
15,633,ноябрь 2022,Васильев Артем Александрович
127,633,февраль 2023,Васильев Артем Александрович
192,633,апрель 2023,Васильев Артем Александрович
241,633,июнь 2023,Васильев Артем Александрович
271,633,июль 2023,Васильев Артем Александрович
298,633,август 2023,Васильев Артем Александрович


In [5]:
fin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             451 non-null    int64 
 1   Причина дубля  301 non-null    object
 2   Ноябрь 2022    156 non-null    object
 3   Декабрь 2022   159 non-null    object
 4   Январь 2023    139 non-null    object
 5   Февраль 2023   145 non-null    object
 6   Март 2023      168 non-null    object
 7   Апрель 2023    174 non-null    object
 8   Май 2023       190 non-null    object
 9   Июнь 2023      190 non-null    object
 10  Июль 2023      195 non-null    object
 11  Август 2023    199 non-null    object
 12  Сентябрь 2023  186 non-null    object
 13  Октябрь 2023   182 non-null    object
 14  Ноябрь 2023    171 non-null    object
 15  Декабрь 2023   146 non-null    object
 16  Январь 2024    95 non-null     object
 17  Февраль 2024   101 non-null    object
 18  Account        451 non-null   

In [6]:
fin_df.head()

,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account
0,42,NaN,"36 220,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
1,657,первая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
2,657,вторая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
3,594,NaN,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
4,665,NaN,"10 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович


In [75]:
# Очистка данных

fin_clean = fin_df.copy()

month_cols = [col for col in fin_clean.columns if col not in ["id", "Причина дубля", "Account"]]

mask_stop = fin_clean.apply(lambda row: any(val in ["стоп", "end"] for val in row[month_cols]), axis=1)
fin_clean = fin_clean[~mask_stop]


for col in month_cols:
    fin_clean[col] = (
        fin_clean[col]
        .apply(lambda x: 0 if x == 'в ноль' else x)
        .str.replace('\xa0', '', regex=False)
        .str.replace(",", ".", regex=False)
        .astype(float)
    )


fin_clean = fin_clean[fin_clean[month_cols].sum(axis=1) != 0]

In [83]:
fin_clean[(fin_clean['id']==15) | (fin_clean['id']==45)]

,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account
151,15,первая часть оплаты,329460.00,329460.00,102433.75,102433.75,102433.75,138158.00,138158.00,102433.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Иванова Мария Сергеевна
152,15,вторая часть оплаты,109820.00,109820.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Иванова Мария Сергеевна
360,45,первая часть оплаты,NaN,NaN,22200.00,22200.00,22200.00,22200.00,22200.00,22200.00,22200.00,22200.00,22200.00,22200.00,22200.00,22200.00,NaN,NaN,Попова Екатерина Николаевна
361,45,вторая часть оплаты,35150.00,47400.00,26930.00,42275.00,49440.00,36795.00,39900.00,39435.00,48555.00,13925.00,62890.00,74480.00,66200.00,55330.00,NaN,NaN,Попова Екатерина Николаевна


In [85]:
fin_agg = (
    fin_clean
    .groupby("id", as_index=False)
    .agg({
        "Account": "first",  
        **{c: "sum" for c in month_cols}  
    })
)

In [89]:
fin_agg.head(20)

,id,Account,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024
0,15,Иванова Мария Сергеевна,439280.00,439280.00,102433.75,102433.75,102433.75,138158.00,138158.00,102433.75,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,31,Васильев Артем Александрович,55100.00,55100.00,0.00,44775.00,44775.00,44775.00,44775.00,44775.00,44775.00,44775.00,44775.00,44775.00,44775.00,44775.00,44775.00,46200.00
2,39,Попова Екатерина Николаевна,137700.00,137700.00,149206.50,149206.50,149206.50,149206.50,149206.50,149206.50,149206.50,149206.50,149206.50,149206.50,149206.50,149206.50,0.00,0.00
3,42,Васильев Артем Александрович,36220.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,45,Попова Екатерина Николаевна,35150.00,47400.00,49130.00,64475.00,71640.00,58995.00,62100.00,61635.00,70755.00,36125.00,85090.00,96680.00,88400.00,77530.00,0.00,0.00
5,46,Смирнова Ольга Владимировна,37939.50,37939.50,39838.50,39298.50,39298.50,39298.50,39298.50,39298.50,39298.50,39298.50,39298.50,39298.50,39298.50,39298.50,0.00,0.00
6,48,Смирнова Ольга Владимировна,0.00,0.00,0.00,39726.00,39726.00,39726.00,0.00,0.00,0.00,0.00,39726.00,39726.00,39726.00,0.00,45175.50,44635.50
7,49,Смирнова Ольга Владимировна,85842.00,85842.00,83358.00,83358.00,83358.00,83358.00,83358.00,83358.00,84582.00,84582.00,84582.00,84582.00,84582.00,84582.00,0.00,0.00
8,54,Смирнова Ольга Владимировна,0.00,0.00,0.00,0.00,41067.00,38461.50,32139.00,31333.50,32139.00,32139.00,49666.50,50094.00,50949.00,50949.00,0.00,0.00
9,55,Иванова Мария Сергеевна,0.00,0.00,0.00,0.00,43173.00,41364.00,44257.50,45868.50,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [90]:
df = prolong_df.merge(fin_df, left_on="project_id", right_on="id", how="inner")

In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725 entries, 0 to 724
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   project_id     725 non-null    int64 
 1   last_month     725 non-null    object
 2   account_name   725 non-null    object
 3   id             725 non-null    int64 
 4   Причина дубля  521 non-null    object
 5   Ноябрь 2022    266 non-null    object
 6   Декабрь 2022   287 non-null    object
 7   Январь 2023    277 non-null    object
 8   Февраль 2023   307 non-null    object
 9   Март 2023      335 non-null    object
 10  Апрель 2023    349 non-null    object
 11  Май 2023       368 non-null    object
 12  Июнь 2023      364 non-null    object
 13  Июль 2023      352 non-null    object
 14  Август 2023    374 non-null    object
 15  Сентябрь 2023  336 non-null    object
 16  Октябрь 2023   328 non-null    object
 17  Ноябрь 2023    314 non-null    object
 18  Декабрь 2023   261 non-null   

In [92]:
df[df['id']==15]

,project_id,last_month,account_name,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account
33,15,декабрь 2022,Иванова Мария Сергеевна,15,первая часть оплаты,"329 460,000","329 460,00","102 433,75","102 433,75","102 433,75","138 158,00","138 158,00","102 433,75",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Иванова Мария Сергеевна
34,15,декабрь 2022,Иванова Мария Сергеевна,15,вторая часть оплаты,"109 820,00","109 820,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Иванова Мария Сергеевна
189,15,февраль 2023,Иванова Мария Сергеевна,15,первая часть оплаты,"329 460,000","329 460,00","102 433,75","102 433,75","102 433,75","138 158,00","138 158,00","102 433,75",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Иванова Мария Сергеевна
190,15,февраль 2023,Иванова Мария Сергеевна,15,вторая часть оплаты,"109 820,00","109 820,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Иванова Мария Сергеевна
239,15,март 2023,Иванова Мария Сергеевна,15,первая часть оплаты,"329 460,000","329 460,00","102 433,75","102 433,75","102 433,75","138 158,00","138 158,00","102 433,75",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Иванова Мария Сергеевна
240,15,март 2023,Иванова Мария Сергеевна,15,вторая часть оплаты,"109 820,00","109 820,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Иванова Мария Сергеевна
270,15,апрель 2023,Иванова Мария Сергеевна,15,первая часть оплаты,"329 460,000","329 460,00","102 433,75","102 433,75","102 433,75","138 158,00","138 158,00","102 433,75",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Иванова Мария Сергеевна
271,15,апрель 2023,Иванова Мария Сергеевна,15,вторая часть оплаты,"109 820,00","109 820,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Иванова Мария Сергеевна
327,15,июнь 2023,Иванова Мария Сергеевна,15,первая часть оплаты,"329 460,000","329 460,00","102 433,75","102 433,75","102 433,75","138 158,00","138 158,00","102 433,75",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Иванова Мария Сергеевна
328,15,июнь 2023,Иванова Мария Сергеевна,15,вторая часть оплаты,"109 820,00","109 820,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Иванова Мария Сергеевна


In [93]:
df.to_csv('merged.csv')